# 📘 Transformer与注意力机制详解

## 🔍 1. 什么是注意力机制？

在深度学习中，**注意力机制**让模型能够动态关注输入序列的相关部分。

例如翻译句子：
> "这只动物没有过马路，因为它太累了"

当判断"它"指代什么时，模型需要**关注**"动物"这个词。

### 核心思想：
不同于传统RNN/Seq2Seq将整个句子编码为固定向量，注意力机制让模型可以**动态权衡**序列中每个词的重要性。

## ✨ 2. 缩放点积注意力（图解版）

### 生活化类比

想象你在书店找书：
- **Query (Q)**：你的需求（"想找Python编程书"）
- **Key (K)**：每本书的目录标题
- **Value (V)**：书籍的实际内容

```mermaid
graph LR
    Q[查询] -->|匹配| K[关键词]
    K -->|生成| W[权重]
    W -->|加权求和| V[值]
    V --> O[输出]
```

### 计算四部曲

1. **相似度计算**：
   ```python
   原始分数 = Q @ K.T  # 矩阵乘法
   ```
   
2. **缩放处理**：
   ```python
   缩放分数 = 原始分数 / sqrt(d_k)  # d_k=键向量维度
   ```
   
3. **权重归一化**：
   ```python
   注意力权重 = softmax(缩放分数)  # 得到0-1之间的权重
   ```
   
4. **加权求和**：
   ```python
   输出 = 注意力权重 @ V  # 最终上下文向量
   ```

### 为什么需要缩放？
当维度`d_k`较大时，点积结果会变得极大，导致softmax梯度消失。缩放保持梯度稳定。

### 数学公式

$$
\text{注意力}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

## 🔁 3. 自注意力机制（完整解析）

### 与传统注意力的区别

| 特性          | 自注意力              | 传统注意力            |
|---------------|----------------------|----------------------|
| 查询来源       | 输入序列自身          | 外部（如解码器）      |
| 键/值来源      | 输入序列自身          | 外部源                |
| 方向性        | 双向                 | 通常单向              |

### 三大核心优势

1. **捕捉长距离依赖**：
   ```text
   句子："这只猫，尽管刚吃过饭还是很饿，大声喵喵叫"
   关系："猫" ↔ "喵喵叫"（距离远但直接关联）
   ```
   
2. **并行计算**：
   所有位置对可以同时计算（不同于RNN的顺序处理）
   
3. **上下文编码**：
   每个词的表示会根据全文语境动态变化

### 计算流程

```python
# 输入词向量 (n × 模型维度)
X = [词1向量, 词2向量, ...]  

# 可学习的投影矩阵
Q = X @ W_Q  # (n × d_k)
K = X @ W_K  # (n × d_k)
V = X @ W_V  # (n × d_v)

# 自注意力计算
注意力 = softmax((Q @ K.T)/sqrt(d_k)) @ V  # (n × d_v)
```

### 实际代码演示

In [ ]:
import torch
import torch.nn.functional as F

# 示例句子编码（3个词，每个词4维）
X = torch.tensor([
    [1.0, 0.2, -0.5, 0.3],  # 词1（"猫"）
    [0.5, 1.2, 0.1, -0.7],  # 词2（"饿"）
    [-0.3, 0.8, 1.1, 0.4]   # 词3（"叫"）
], dtype=torch.float32)

# 随机初始化投影矩阵（实际训练中自动学习）
W_Q = torch.randn(4, 3)
W_K = torch.randn(4, 3)
W_V = torch.randn(4, 2)

# 计算Q,K,V
Q = X @ W_Q
K = X @ W_K
V = X @ W_V

# 自注意力计算
d_k = Q.size(-1)
分数 = (Q @ K.T) / torch.sqrt(torch.tensor(d_k))
权重 = F.softmax(分数, dim=-1)
输出 = 权重 @ V

print("注意力权重矩阵：\n", 权重)
print("\n输出（上下文感知的词向量）：\n", 输出)

### 结果解读
- **注意力权重**：显示词与词之间的关注强度
- **输出向量**：每个词的新表示都融合了相关上下文信息

> 注意：实际应用会使用多头注意力（Multi-Head Attention）

## 🧠 4. Transformer架构（编码器-解码器）

原始Transformer组成：

### 编码器模块：
- 多头自注意力
- 前馈神经网络
- 残差连接 + 层归一化

### 解码器模块：
- 掩码多头自注意力
- 编码器-解码器注意力
- 前馈神经网络
- 残差连接 + 层归一化

```mermaid
graph TD
    A[输入] --> B[位置编码]
    B --> C[编码器堆叠]
    C --> D[上下文向量]
    D --> E[解码器堆叠]
    E --> F[输出]
```

## 🧩 5. 多头注意力机制

Transformer使用**多组注意力头**而非单个：

$$
\text{多头注意力}(Q, K, V) = \text{拼接}(头_1, ..., 头_h)W^O
$$

其中每个头的计算：
$$
头_i = \text{注意力}(QW_i^Q, KW_i^K, VW_i^V)
$$

### 多头设计的优势
- 不同头学习不同关注模式
- 例如可能分别关注：
  - 语法结构
  - 语义关系
  - 指代关联
- 类似CNN中多滤波器的效果

## 🔄 6. 逐位置前馈网络

序列每个位置独立通过相同的前馈网络：

$$
\text{FFN}(x) = \text{ReLU}(xW_1 + b_1)W_2 + b_2
$$

### 关键特性：
- 独立处理每个位置
- 通常先扩维再降维（如512 → 2048 → 512）
- 提供额外非线性变换
- 又称"位置感知"前馈网络

## 🌀 7. 位置编码

因Transformer没有循环结构，需添加**位置编码**到输入嵌入：

$$
PE_{(位置, 2i)} = \sin\left(\frac{位置}{10000^{2i/d_{模型}}}\right)
$$
$$
PE_{(位置, 2i+1)} = \cos\left(\frac{位置}{10000^{2i/d_{模型}}}\right)
$$

### 可视化模式：
<img src="https://jalammar.github.io/images/t/transformer_positional_encoding_example.png" width="400">

### 设计原理：
- 每个位置有唯一编码
- 相对位置可被线性关注
- 对任意长度序列都适用

## 💡 8. 自注意力完整实现

In [ ]:
import math
import torch
import torch.nn as nn

class 自注意力(nn.Module):
    def __init__(self, 模型维度, 键维度):
        super().__init__()
        self.W_Q = nn.Linear(模型维度, 键维度)  # 查询变换
        self.W_K = nn.Linear(模型维度, 键维度)  # 键变换
        self.W_V = nn.Linear(模型维度, 键维度)  # 值变换
        self.键维度 = 键维度
        
    def forward(self, X):
        Q = self.W_Q(X)  # (批大小, 序列长, 键维度)
        K = self.W_K(X)  # (批大小, 序列长, 键维度)
        V = self.W_V(X)  # (批大小, 序列长, 键维度)
        
        分数 = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.键维度)
        权重 = torch.softmax(分数, dim=-1)
        输出 = torch.matmul(权重, V)
        
        return 输出, 权重

# 使用示例
模型维度 = 512
键维度 = 64
批大小 = 4
序列长 = 10

注意力层 = 自注意力(模型维度, 键维度)
输入 = torch.rand(批大小, 序列长, 模型维度)
输出, 注意力权重 = 注意力层(输入)

print(f"输入维度: {输入.shape}")
print(f"输出维度: {输出.shape}")
print(f"注意力权重维度: {注意力权重.shape}")

## 🏁 总结

- **注意力机制**：动态聚焦输入相关部分
- **自注意力**：捕捉序列元素间的两两关系
- **Transformer架构**：
  - 编码器：通过自注意力处理输入
  - 解码器：生成式预测输出
- **关键创新**：
  - 多头注意力
  - 位置编码
  - 残差连接
- **影响**：BERT、GPT等模型的基石

后续方向：
- 实验多头注意力
- 实现完整Transformer
- 探索变体（稀疏注意力、线性注意力）